In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

import time

In [7]:
class Estadao:

    def __init__(self, base_url='https://www.estadao.com.br/'):
        self.base_url = base_url
    
    def yesterday_url_str(self):
        yesterday = datetime.now() - timedelta(1)
        yesterday_str = yesterday.strftime("%d/%m/%Y")
        return f'/noticia{yesterday_str}'

    def getUrls(self, url):
        try:
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            time.sleep(5)
            
            page_source = driver.page_source
            driver.quit()
            
            soup = BeautifulSoup(page_source, 'html.parser')
            all_hrefs = soup.find_all('a', class_='image') + soup.find_all('a', class_='news-link')
            urls = []
            for a in all_hrefs:
                href = a.get('href')
                if href:
                    urls.append(href)
            
            return list(set(urls)) 
        except:
            return []



    def getNewsData(self, url, only_yesterday=True):
        try:
            browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(url, headers = browsers)
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')

            title = soup.find('h1').get_text()
            timestamp = soup.find('time').get_text()
            isYesterday = self.yesterday_url_str() in timestamp
            
            content = "" 
            newsBody = soup.find('div', class_="news-body")
            for p in newsBody.find_all('p'):
                content += p.get_text()
                content += "\n"
            
            content = re.sub(r'\s+', ' ', content).strip()
            
            if only_yesterday and not isYesterday:
                return None

            return {
                "url": url,
                "title": title,
                "content": content
            }
        except:
            return None
